# Instance Segmentation Pipeline using Remo

In this tutorial, we will use Remo to accelerate and improve the process of building a transfer learning pipeline for an Instance Segmentation task.




In particular, we will:

* Use Remo to browse through our images and annotations
* Use Remo to understand the properties of the dataset and annotations by  visualizing statistics.
* Create a custom train, test, valid split in-place using Remo image tags.
* Fine tune a pre-trained MaskRCNN model from Detectron2 and do some inference
* Visually compare Mask predictions with the ground truth



**Along the way, we will see how the Dataset visualization provided Remo helps to gather insights to improve the dataset and the model.**

Before proceeding with the tutorial, we would need to install the required dependencies. This can be done by executing the next cell.

Once complete, **restart your runtime**, to ensure that the installed packages can be detected.



In [ ]:
!pip install imantics
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [1]:
from google.colab import drive
GDRIVE_ROOT = "/gdrive"
drive.mount(GDRIVE_ROOT)

Mounted at /gdrive


In [ ]:
!pip install "/gdrive/My Drive/remo-0.5.2.1-py3-none-any.whl"
!python -m remo_app init --colab

Let us then import the required packages.

In [42]:
import remo
remo.set_viewer('jupyter')

import numpy as np
import os
from PIL import Image
import glob
import random
random.seed(4)

from imantics import Polygons, Mask

import torch, torchvision

# Detectron 2 files
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances

### Adding Data to Remo

* The Dataset used is a subset of the MS-COCO Dataset.
* The directory structure of the dataset is as follows
```
├── instance_segmentation_dataset
    ├── images
        ├── image_1.jpg
        ├── image_2.jpg
        ├── ...
    ├── annotations
        ├── dataset_annotations.json
```


In [ ]:
if not os.path.exists('object_detection_dataset.zip'):
  !unzip instance_segmentation_dataset.zip
else:
  print('Files already downloaded')

In [46]:
path_to_annotations = 'instance_segmentation_dataset/annotations/'
path_to_images = 'instance_segmentation_dataset/images/'

## Train / test split
In Remo, we can use tags to organise our images. Among other things, this allows us to generate train / test splits without the need to move image files around.

To do this, we just need to pass a dictionary (mapping tags to the relevant images paths) to the function ```remo.generate_image_tags()```.

In [47]:
im_list = [i for i in glob.glob(path_to_images + '/**/*.jpg', recursive=True)]
im_list = random.sample(im_list, len(im_list))

train_idx = round(len(im_list) * 0.4)
valid_idx = train_idx + round(len(im_list) * 0.3)
test_idx  = valid_idx + round(len(im_list) * 0.3)

tags_dict =  {'train' : im_list[0:train_idx], 
              'valid' : im_list[train_idx:valid_idx], 
              'test' : im_list[valid_idx:test_idx]}

train_test_split_file_path = os.path.join(path_to_annotations, 'images_tags.csv') 
remo.generate_image_tags(tags_dictionary  = tags_dict, 
                         output_file_path = train_test_split_file_path, 
                         append_path = False)

'instance_segmentation_dataset/annotations/images_tags.csv'

### Create a dataset

To create a dataset we can use ```remo.create_dataset()```, specifying the path to data and annotations.

The class encoding (if required) is passed via a dictionary.

For a complete list of formats supported, you can <a href="https://remo.ai/docs/annotation-formats/"> refer to the docs</a>.


In [ ]:
instance_segmentation_dataset = remo.create_dataset(name = 'instance_segmentation_dataset', local_files = [path_to_annotations, path_to_images], annotation_task='Instance Segmentation')

**Visualizing the dataset**

To view and explore images and labels, we can use Remo directly from the notebook. We just need to call ```dataset.view()```.

In [ ]:
instance_segmentation_dataset.view()

**Dataset Statistics**

Using Remo, we can quickly visualize some key Dataset properties that can help us with our modelling, without needing to write extra boilerplate code.

This can be done either from code, or using the visual interface.

In [ ]:
instance_segmentation_dataset.get_annotation_statistics()

In [ ]:
instance_segmentation_dataset.view_annotation_stats()

**Exporting the dataset**

To export annotations according to the train, test split in a format accepted by the model, we use the ```dataset.export_annotations_to_file()``` method, and filter by the desired tag.

For a complete list of formats supported, you can <a href="https://remo.ai/docs/annotation-formats/"> refer to the docs</a>.

In [53]:
path_to_train = path_to_annotations + "dog_train.json"
path_to_test = path_to_annotations + "dog_test.json"

In [54]:
instance_segmentation_dataset.export_annotations_to_file(path_to_train, annotation_format='coco', filter_by_tags=['train'], export_tags=False, append_path=False)
instance_segmentation_dataset.export_annotations_to_file(path_to_test, annotation_format='coco', filter_by_tags=['test'], export_tags=False, append_path=False)

# Detectron2

Here we will start working with the ```Detectron2``` framework written in PyTorch.

## Feeding Data into Detectron2

First, you are required to register your dataset, Detectron2 assumes your data is in MS-COCO format.

The ```register_coco_instances``` method takes in the following parameters:

* **path_to_annotations:** Path to annotation files. Format: COCO JSON.
* **path_to_images:** Path to the folder containing the images.

After this, we store the metadata for future operations.

In [60]:
register_coco_instances('dog_trainn', {}, path_to_train, path_to_images)
register_coco_instances('dog_testt', {}, path_to_test, path_to_images)

train_metadata = MetadataCatalog.get('dog_train')

To visualise the labels as strings rather than IDs, we can use a dictionary mapping the two of them.

In [61]:
mapping = {k: v for k, v in enumerate(train_metadata.thing_classes)}

## Training the Model

In this tutorial, we use a ```Mask RCNN``` architecture with a ```ResNet-50 Backbone```, pre-trained on on COCO train2017. This is loaded directly from Detectron2.

To train the model, we specify the following details:

- **model_yaml_path:** Configuration file for the Mask RCNN model.
- **model_weights_path**: Symbolic link to the desired Mask RCNN architecture.

The parameters can be tweaked by overriding the correspodning variable in the ```cfg```.

In [ ]:
model_yaml_path = './detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'
model_weights_path = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl'

cfg = get_cfg()
cfg.merge_from_file(model_yaml_path)
cfg.DATASETS.TRAIN = ('dog_train',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_weights_path # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = 150    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (data, fig, hazelnut)

**Instantiating the Trainer**

The trainer is instantiated with the required configuration, and the training process is started.

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Visualizing Predictions

Using Remo, we can easily iterate through the images to compare the model predictions against the original labels.

To do this, we just need to upload the model predictions to a new ```AnnotationSet```, which we call `model_predictions`

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("dog_test", )
predictor = DefaultPredictor(cfg)
test_dataset_dicts = DatasetCatalog.get("dog_test")

In [38]:
for d in test_dataset_dicts:    
    im = np.array(Image.open(d["file_name"]))
    outputs = predictor(im)
    pred_classes = outputs['instances'].get('pred_classes').cpu().numpy()
    masks = outputs['instances'].get("pred_masks").cpu().permute(1, 2, 0).numpy()
    image_name = d['file_name']
    annotations = []
    
    if masks.shape[2] != 0:
        for i in range(masks.shape[2]):
            polygons = Mask(masks[:, :, i]).polygons()
            annotation = remo.Annotation()
            annotation.img_filename = image_name
            annotation.classes = mapping[pred_classes[i]]
            annotation.segment = polygons.segmentation[0]
            annotations.append(annotation)
    else:
        polygons = Mask(masks[:, :, 0]).polygons()
        annotation = remo.Annotation()
        annotation.img_filename = image_name
        annotation.classes = mapping[pred_classes[0]]
        annotation.segment = polygons.segmentation[0]
        annotations.append(annotation)

In [26]:
model_predictions = instance_segmentation_dataset.create_annotation_set(annotation_task = 'Instance Segmentation', name = 'model_predictions')

instance_segmentation_dataset.add_annotations(annotations, annotation_set_id=model_predictions.id)

Progress 100% - 1/1 - elapsed 0:00:00.001000 - speed: 1000.00 img / s, ETA: 0:00:00
Acquiring data - completed                                                                           
Processing data - completed                                                                          
Data upload completed


In [ ]:
instance_segmentation_dataset.view()

By visualizing the predicted masks against the ground truth, we can go past summary performance metrics, and visually inspect model biases and iterate to improve it.